###### 2020-01-25

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [61]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pandas as pd
import time

# 1. '에펨 코리아' 크롤러

In [80]:
path = './크롬드라이버/chromedriver'
driver = webdriver.Chrome(path)

time.sleep(3)

In [ ]:
main_url = 'https://www.fmkorea.com/index.php?mid=best&page='
url_page_list = list(range(5, 7))


comment_data = []

for url_page in url_page_list:
    url = main_url + str(url_page)
    
    driver.get(url)
    
    time.sleep(2)
    
    
    post_list = driver.find_elements_by_css_selector('div.fm_best_widget > ul > li')

    for idx in range(len(post_list)):
        
        print('현재 페이지 : {} / 현재 게시물번호 : {}' .format(url_page, idx+1))
        post_list[idx].find_element_by_css_selector('div > a').click()

        time.sleep(2)


        # 댓글 페이지를 담는 리스트
        page_list = driver.find_elements_by_css_selector('div#cmtPosition > div.bd_pg > a')
        page_list = page_list[1:len(page_list)-1]
        
        max_page = len(page_list)
        k = 1
        while True :

            time.sleep(2)

            # 댓글을 담고있는 리스트
            comment_list = driver.find_elements_by_css_selector('div#cmtPosition > ul.fdb_lst_ul > li')

            # 댓글 크롤링
            for comment in comment_list:
                crawled_comment = comment.find_elements_by_css_selector('li > div')[1].text
                comment_data.append(crawled_comment)
                
            if (k == max_page+1) | (not page_list):
                break
            else :
                # 댓글 페이지 순차적으로 이동
                page_list = driver.find_elements_by_css_selector('div#cmtPosition > div.bd_pg > a')
                page_list = page_list[1:len(page_list)-1]
                
                for element in page_list :
                    if element.text == str(k):
                        page = element
                page.send_keys('\n')
                k += 1
                
                
        print('현재 크롤링한 데이터수 : {}' .format(len(comment_data)))
        driver.back()

        time.sleep(2)
        
        # 뒤돌아 가기하면 'post_list'가 초기화 된다(없어져버린다)
        # 그래서 다시 지정해 주어 가장 밖의 for문에서 에러가 나지 않도록 한다.
        post_list = driver.find_elements_by_css_selector('div.fm_best_widget > ul > li')


현재 페이지 : 5 / 현재 게시물번호 : 1
현재 크롤링한 데이터수 : 85
현재 페이지 : 5 / 현재 게시물번호 : 2
현재 크롤링한 데이터수 : 235
현재 페이지 : 5 / 현재 게시물번호 : 3
현재 크롤링한 데이터수 : 328
현재 페이지 : 5 / 현재 게시물번호 : 4
현재 크롤링한 데이터수 : 467
현재 페이지 : 5 / 현재 게시물번호 : 5
현재 크롤링한 데이터수 : 517
현재 페이지 : 5 / 현재 게시물번호 : 6
현재 크롤링한 데이터수 : 567
현재 페이지 : 5 / 현재 게시물번호 : 7
현재 크롤링한 데이터수 : 651
현재 페이지 : 5 / 현재 게시물번호 : 8
현재 크롤링한 데이터수 : 766
현재 페이지 : 5 / 현재 게시물번호 : 9
현재 크롤링한 데이터수 : 872
현재 페이지 : 5 / 현재 게시물번호 : 10
현재 크롤링한 데이터수 : 987
현재 페이지 : 5 / 현재 게시물번호 : 11
현재 크롤링한 데이터수 : 1032
현재 페이지 : 5 / 현재 게시물번호 : 12
현재 크롤링한 데이터수 : 1132
현재 페이지 : 5 / 현재 게시물번호 : 13
현재 크롤링한 데이터수 : 1167
현재 페이지 : 5 / 현재 게시물번호 : 14
현재 크롤링한 데이터수 : 1209
현재 페이지 : 5 / 현재 게시물번호 : 15
현재 크롤링한 데이터수 : 1307


In [73]:
page_list = driver.find_elements_by_css_selector('div#cmtPosition > div.bd_pg > a')
page_list = page_list[1:len(page_list)-1]

In [74]:
page_list[0].text

'1'

In [12]:
data = {'txt' : comment_data}
add_review_df = pd.DataFrame(data)

add_review_df.drop_duplicates(keep='first', inplace=True)

# 2. 크롤링한 리뷰 데이터 전처리

In [13]:
stop_words =  '''◆ ■ ★ = ● ※ ▷ ▶ ▲ ● ~ ◀ \ ＆ # : @ ○ ▽ ♤ [ ] * ♡ + { } ♬ ♪ ^ ◈ _ . ↑ ◎ ♥ ↔ → ♩ ← ☆ ↓ - ◁ △ ▼ ♠ ◀ ◑ ◐ ↔ ♣ ＠ 〓 % ▣ □ ( ) \' \" ＃ ⊙ § ; ◇ | ` $ ♧ ટૂ ○ & / ! < > ? , 「 」 ㆍ ？ ℵ ’ ‘ “ ” ↗ ， ！ · † Ф 十 ど デ タ ベ は 結 論 を 出 す こ と が で き な い 。 い つ も や ね ぇ 故 山 ‥ 有 点 童 年 的 味'''
stop_words_list = stop_words.split(' ')

for stop_words in stop_words_list : 
    add_review_df['txt'] = add_review_df['txt'].str.replace(stop_words, '')

In [14]:
# 의미가 존재하는 기호를 사전형태로 의미를 정리한다.
substitution_dict = {}

############ 아스키코드 규칙성을 보이는 특수기호들
# 1. ㄱ / ㄴ / ㅓ / ㅏ
temp_list01 = [(chr(element), 'ㄱ') for element in range(9488, 9491+1)]
temp_list02 = [(chr(element), 'ㄴ') for element in range(9492, 9495+1)]
temp_list03 = [(chr(element), 'ㅏ') for element in range(9500, 9507+1)]
temp_list04 = [(chr(element), 'ㅓ') for element in range(9508, 9516+1)]
weird_symbol_list = temp_list01 + temp_list02 + temp_list03 + temp_list04

for weird_symbol, substitution in weird_symbol_list:
    substitution_dict[weird_symbol] = substitution

# 2. 자음
substitution_dict['⊂'] = 'ㄷ'
substitution_dict['Ø'] = 'ㅇ'
substitution_dict['人'] = 'ㅅ'
substitution_dict['乙'] = 'ㄹ'
substitution_dict['ス'] = 'ㅈ'
substitution_dict['て'] = 'ㄷ'

# 3. 아랍어
# સ
substitution_dict['ગ'] = '이'
substitution_dict['લ'] = '더'
substitution_dict['નુ'] = '엉'
substitution_dict['ન'] = '어'
substitution_dict['ㄐ'] = '니'
substitution_dict['પ'] = '니'
substitution_dict['શ'] = '리'
substitution_dict['ય'] = '러'
substitution_dict['સ'] = '러'

# 4. 알파벳
substitution_dict['α'] = 'a'
substitution_dict['ɕ'] = 'c'
substitution_dict['с'] = 'c'

# 5. 한글 자음
korean_ja_eum_list = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
substitution_ja_eum01 = [ chr(element) for element in range(ord('㈀'), ord('㈍')+1) ]
substitution_ja_eum02 = [ chr(element) for element in range(ord('㉠'), ord('㉭')+1) ]

for real_ja_eum, bracket_ja_eum, circle_ja_eum in zip(korean_ja_eum_list, substitution_ja_eum01, substitution_ja_eum02):
    substitution_dict[bracket_ja_eum] = real_ja_eum
    substitution_dict[circle_ja_eum]  = real_ja_eum

# 6. 한글 음절 (아)
korean_syllable_list = ['가', '나', '다', '라', '마', '바', '사', '아', '자', '차', '카', '타', '파', '하']
substitution_syllable01 = [ chr(element) for element in range(ord('㈎'), ord('㈛')+1) ]
substitution_syllable02 = [ chr(element) for element in range(ord('㉮'), ord('㉻')+1) ]

for real_syllable, bracket_syllable, circle_syllable in zip(korean_syllable_list, substitution_syllable01, substitution_syllable02):
    substitution_dict[bracket_syllable] = real_syllable
    substitution_dict[circle_syllable]  = real_syllable


# 7. 숫자
number_list = list(map(str, range(1, 21)))
substitution_num01 = [ chr(element) for element in range(ord('⑴'), ord('⒇')+1) ]
substitution_num02 = [ chr(element) for element in range(ord('①'), ord('⑳')+1) ]

for num, bracket_num, circle_num in zip(number_list, substitution_num01, substitution_num02):
    substitution_dict[bracket_num] = num
    substitution_dict[circle_num]  = num

# 8. 또다른 숫자
number_list = list(map(str, range(1, 10)))
substitution_num = [ chr(element) for element in range(ord('１'), ord('９')+1) ]

for num, symbol_num in zip(number_list, substitution_num):
    substitution_dict[symbol_num] = num

# 9. 알파벳
lower_alphabet_list = list('abcdefghijklmnopqrstuvwxyz')
circle_lower_alphabet_list = [ chr(element) for element in range(ord('ⓐ'), ord('ⓩ')+1) ]

for lower_alphabet, circle_lower_alphabet in zip(lower_alphabet_list , circle_lower_alphabet_list):
    substitution_dict[circle_lower_alphabet] = lower_alphabet


    
######################## 아스키코드 규칙성을 지키지 않는 특수문자 
# 10. 아스키 규칙성을 보이지 않는 문자 
substitution_dict['盧'] = '노'
substitution_dict['乂'] = 'x'
substitution_dict['ㄋ'] = '3'
substitution_dict['З'] = '3'
substitution_dict['э'] = '3'
substitution_dict['б'] = '6'
substitution_dict['Э'] = '3'
substitution_dict['з'] = '3'
substitution_dict['м'] = '3'
substitution_dict['ё'] = 'e'
substitution_dict['Н'] = 'H'
substitution_dict['Р'] = 'P'
substitution_dict['β'] = 'b'
substitution_dict['Д'] = 'A'
substitution_dict['ß'] = 'b'
substitution_dict['я'] = 'r'
substitution_dict['丅'] = 'T'
substitution_dict['д'] = 'A'
substitution_dict['ℳ'] = 'M'
substitution_dict['р'] = 'p'
substitution_dict['М'] = 'M'
substitution_dict['古'] = '고'
substitution_dict['典'] = '전'
substitution_dict['И'] = 'N'
substitution_dict['ί'] = 'i'
substitution_dict['Я'] = 'R'
substitution_dict['у'] = 'y'
substitution_dict['Ɔ'] = 'C'
substitution_dict['ф'] = 'o'
substitution_dict['⻏'] = 'B'
substitution_dict['℘'] = 'G'
substitution_dict['Ｏ'] = 'o'
substitution_dict['Щ'] = 'w'
substitution_dict['С'] = 'c'
substitution_dict['無'] = '무'
substitution_dict['間'] = '간'
substitution_dict['道'] = '도'
substitution_dict['₂'] = '2'
substitution_dict['日'] = '일'
substitution_dict['й'] = 'N'
substitution_dict['Ｈ'] = 'H'
substitution_dict['Ｃ'] = 'C'
substitution_dict['ｏ'] = 'o'

In [15]:
for target, substitution in substitution_dict.items():
    add_review_df['txt'] = add_review_df['txt'].str.replace(target, substitution)

In [16]:
lower_alphabet_list = list('abcdefghijklmnopqrstuvwxyz')
upper_alphabet_list = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

for upper, lower in zip(upper_alphabet_list, lower_alphabet_list):
    add_review_df['txt'] = add_review_df['txt'].str.replace(upper, lower)

In [17]:
print('Before : ', add_review_df.shape)

index1 = add_review_df['txt'] == ''
index2 = add_review_df['txt'] == ' '
index3 = add_review_df['txt'] == '  '

index = index1 | index2 | index3
add_review_df = add_review_df[~index]

print('After : ', add_review_df.shape)

Before :  (4534, 1)
After :  (4531, 1)


#### 데이터라벨링을 위해 소분화 저장
   - 영화관련된 내용의 문장인지 아닌지 라벨링 부여

In [18]:
lookback = 500
mok = len(add_review_df) // lookback


for i in range(mok):
    temp_df = add_review_df.iloc[i*500 : 500*i+500, ]
    temp_df.to_csv('./add_commentdf_' + str(i+1) + '.csv', encoding='UTF-8-SIG')
add_review_df[mok*500:].to_csv('./add_comment_df_' + str(mok) +'.csv', encoding='UTF-8-SIG')